# Setup

### Dependancies:
- Sage Math 9.8+ (https://www.sagemath.org/index.html)
- Python 3.11 +

```bash
sage -pip install cryptography
sage -pip install web3
sage -pip install hexbytes
```

### Encryption

In [9]:
from sage.all import *

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os

class AES:
    def encrypt(message, key):
        # Ensure the key length is valid for AES (16, 24, 32 bytes)
        if len(key) not in [16, 24, 32]:
            raise ValueError("Key must be either 16, 24, or 32 bytes long.")
        
        # Pad the message to make it a multiple of the block size (16 bytes for AES)
        padder = padding.PKCS7(algorithms.AES.block_size).padder()
        padded_message = padder.update(message.encode()) + padder.finalize()
        
        # Generate a random initialization vector (IV)
        iv = os.urandom(16)
        
        # Create an AES cipher object with the given key and a randomly generated IV
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()
        
        # Encrypt the padded message
        encrypted_message = encryptor.update(padded_message) + encryptor.finalize()
        
        return iv + encrypted_message

    def decrypt(encrypted_message, key):
        # Extract the IV, which is the first 16 bytes of the encrypted message
        iv = encrypted_message[:16]
        ciphertext = encrypted_message[16:]
        
        # Create an AES cipher object with the given key and the extracted IV
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()
        
        # Decrypt the ciphertext
        decrypted_padded_message = decryptor.update(ciphertext) + decryptor.finalize()
        
        # Unpad the message to retrieve the original plaintext
        unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
        decrypted_message = unpadder.update(decrypted_padded_message) + unpadder.finalize()
        
        return decrypted_message.decode()
        
    def key_size(lam):
        # 256 bits is sufficient for good symetric encryption, if `lam < 256`, we limit k to 256 bits
        # This function finds the suitable key size
        return 256 if lam > 256 else lam

In [3]:
key = os.urandom(32)  # Generate a random 32-byte key
message = 'Hello, this is a secret message!'
encrypted = AES.encrypt(message, key)
print("Encrypted:", encrypted)

decrypted = AES.decrypt(encrypted, key)
print("Decrypted:", decrypted)

Encrypted: b'D!(U\x88gg{0h\x9c\xf8\xb9 \x91)\xaf\x12\xe9\tiC\x12`kG\xf8uG7\x99V\x8b\xa0\xb9\xad\xbct\xaaM\x85K\x08\x04B\xff\x19\x00y!"p\xd8\x0f,\x84H\xdf\x94y bW\x98'
Decrypted: Hello, this is a secret message!


### Hashing

In [4]:
import hashlib

class SHA2:
    @staticmethod
    def hash(message):
        """
        Generate a SHA-256 hash of the given message.

        Args:
        message (str): The message to hash.

        Returns:
        str: The hexadecimal string representing the hash of the message.
        """
        # Ensure the message is in bytes, encode if it's not
        if isinstance(message, str):
            message = message.encode()

        # Create a new SHA-256 hash object
        hash_object = hashlib.sha256()
        hash_object.update(message)

        # Return the hexadecimal digest of the hash
        return hash_object.hexdigest()

In [5]:
# Example usage
message = "Hello, this is a secret message!"
hash_output = SHA2.hash(message)
print("SHA-256 Hash:", hash_output)

SHA-256 Hash: b17bc84328a218501c955d184899513826cb4ec53466fc1c836d742ea8a94fe2


### Serialisation| 

In [6]:
import json

def pack_dict(d):
    
    # Serialize data to JSON
    packed_data = json.dumps(d)
    return packed_data

def unpack_dict(packed_d):
    # Deserialize data from JSON
    d = json.loads(packed_d)
    return d

In [7]:
# Example usage
data = {"a": "Hello!", "b": int(1), "c": int(55)}
packed = pack_dict(data)
print("Packed Data: {}".format(packed))
unpacked = unpack_dict(packed)
assert data == unpacked, "Issue in packing/unpacking data"
print("Unpacked Data: {}".format(unpacked))

Packed Data: {"a": "Hello!", "b": 1, "c": 55}
Unpacked Data: {'a': 'Hello!', 'b': 1, 'c': 55}


# RSA TLP

In [8]:
class TLP:
    def setup(lam, delta, S):
        # Generate two large prime numbers q1 and q2
        q1 = random_prime(2^(lam//2), proof=False)
        q2 = random_prime(2^(lam//2), proof=False)
        while q1 == q2:
            q2 = random_prime(2^(lam//2), proof=False)
        
        # Compute N and Euler's totient function phi(N)
        N = q1 * q2
        phi_N = (q1 - 1) * (q2 - 1)
        
        # Generate a symmetric key k (for simplicity, using a random integer)
        k = ZZ.random_element(2^AES.key_size(lam)) # IMPORTANT: This is an insecure random
        
        # Choose a uniformly random value r from Z_N*
        r = ZZ.random_element(1, N)
        
        # Compute a as 2^T mod phi(N)
        T = S * delta
        a = power_mod(2, T, phi_N)
        
        # Define the public key and secret key
        pk = (N, T, r)
        sk = (q1, q2, a, k)
        
        return pk, sk
    
    def gen_puzzle(m, pk, sk):
        # Step 1(a): encrypt the message under key k using symmetric-key encryption
        N, T, r = pk
        q1, q2, a, k = sk
        p1 = AES.encrypt(m, int(k).to_bytes(AES.key_size(lam) // 8, byteorder='big'))
        
        # Step 1(b): encrypt the symmetric-key encryption key k
        p2 = k + power_mod(r, a, N)
        
        # Step 1(c): set p as puzzle and output the puzzle
        puzzle = (p1, p2)
        return puzzle
    
    def solve(p, pk):
        # Step 3: Solve the puzzle
        N, T, r = pk
        p1, p2 = p
    
        b = power_mod(r, 2^T, N)
        k = p2 - b
        m = AES.decrypt(p1, int(k).to_bytes(AES.key_size(lam) // 8, byteorder='big'))
        
        # Return the solution
        return m
    
    



### Example usage:

In [100]:
lam = 400  # Security parameter (use 2048 or more in real scenarios)
delta = 5  # The message should remain hidden for 1 hour
S = 10^4  # Maximum number of squaring modulo N per second

# Setup the puzzle
pk, sk = TLP.setup(lam, delta, S)
print("Public Key:", pk)
print("Secret Key:", sk)

# Example message and keys (generated previously)
message = "Hello!"

print("Challenge message: {}".format(message))

# Generate the puzzle
puzzle = TLP.gen_puzzle(message, pk, sk)

# Solve the puzzle
solution = TLP.solve(puzzle, pk)

# Validate the solution
assert solution == message, "The solution is incorrect."

# The solution should match the original message
print("Correctly recovered message: {}".format(solution))



Public Key: (510025372896405503562181939844861803695909449689006843541472280371079003562081595756592899742130010405863936375040844579, 50000, 508429351764106695272054776412045927295235932839466890483140697064487842756381042198353395373645577652849089048801355000)
Secret Key: (830632509962090421719728243199490829210096206688212000315143, 614020480512715257236763821904418931841428411702556404994053, 172288291005955329040895582872573812747564762639816934048763601233798204510734438883691706238007551976287524483695945456, 95149176508045457793713760977100126731466746030745455817895881364197409923689)
Challenge message: Hello!
Correctly recovered message: Hello!


# C-TLP

In [812]:
class TimeLockedPuzzles:
    def __init__(self, lam, delta, z):
        # lam: security parameter
        # delta: the array of time intervals
        # z: number of puzzles

        self.z = z
        self.delta = delta

        # Assume TLP.Setup function exists and initializes the cryptographic primitives
        self.pk, self.sk = self.TLP_setup(lam, delta, z)

    def TLP_setup(self, lam, delta, z, S=10**4):
        # (a) Compute N = q1 * q2 where q1 and q2 are large randomly chosen prime numbers
        q1 = random_prime(2^(lam // 2), proof=False)
        q2 = random_prime(2^(lam // 2), proof=False)
        while q1 == q2:  # Ensure q1 and q2 are different
            q2 = random_prime(2^(lam // 2), proof=False)
        N = q1 * q2
        phi_N = (q1 - 1) * (q2 - 1)

        # (b) Set Tj as the total number of squaring needed
        # For this example, let's assume delta is in seconds and we want Tj to represent squaring for the duration delta[j]
        # Adjust this as per your security needs and system performance
          # Assuming a high number of squarings per second for the example
        T = S * delta

        # (c) Compute values aj for all j
        a = power_mod(2, T, phi_N)

        # (d) Choose z fixed-size random generators rj, and z random keys kj for symmetric-key encryption
        # We'll use a simple method to choose random rj and kj here.
        r = [ZZ.random_element(2, N) for _ in range(1, z)]
        k = [ZZ.random_element(2, N) for _ in range(1, z)]

        # (e) Pick random d values for commitments
        d = [ZZ.random_element(2, N) for _ in range(z)]

        # (f) Set the public key and secret key
        aux = 'aux'  # Placeholder for cryptographic hash function's description and size of random values
        pk = {'N': N, 'T': T, 'r': r[0], 'aux': aux}  # Only the first r is public
        sk = {'q1': q1, 'q2': q2, 'a': a, 'k': k, 'r': r, 'd': d}

        return pk, sk

    def gen_puzzle(self, messages):
        # Encrypt the messages and generate puzzles
        self.puzzles = []
        self.commitments = []

        # Encrypt the messages and generate puzzles
        for j in range(self.z, 0, -1):

            # Special handling for j = 1
            # if j == 1:
            #     r_j = self.pk['r']
            # else:
            r_j = self.sk['r'][j - 2]


            pkj = (self.pk['N'], self.pk['T'], r_j)
            skj = (self.sk['q1'], self.sk['q2'], self.sk['a'], self.sk['k'][j - 2])

            # If j == z, we do not include the 'r_j+1' value
            if j == self.z:
                m_enc = pack_dict({"message": messages[j - 1], "d_j": int(self.sk['d'][j - 1])})
            else:
                m_enc = pack_dict({"message": messages[j - 1], "d_j": int(self.sk['d'][j - 1]), "r_j": int(self.sk['r'][j - 1])})
                        
            puzzle = TLP.gen_puzzle(m_enc, pkj, skj)
            self.puzzles.append(puzzle)

            # Commit to each message with a cryptographic hash function
            m_com = {"message": messages[j - 1], "d_j": int(self.sk['d'][j - 1])}
            commitment = SHA2.hash(pack_dict(m_com))  # Placeholder for cryptographic hash function

        
            self.commitments.append(commitment)

        self.commitments = list(reversed(self.commitments))
        self.puzzles = list(reversed(self.puzzles))
        return self.puzzles, self.commitments

    def solve_puzzle(self):
        # Decrypt the puzzles
        solutions = []
        u = None

        for j, puzzle in enumerate(self.puzzles, start=1):
            # Special handling for j = 1
            if j == 1:
                r_j = self.pk['r']
            else:
                r_j = u
                

            # Set pkj
            pkj = (self.pk['N'], self.pk['T'], r_j)


            # Solve the puzzle
            xj = TLP.solve(puzzle, pkj)


            # Parse the solution
            m_unpacked = unpack_dict(xj)
            if j < z:
                mj, dj, rj_plus_one = m_unpacked["message"], m_unpacked["d_j"],  m_unpacked["r_j"]
                u = rj_plus_one
            else:
                mj, dj = m_unpacked["message"], m_unpacked["d_j"]


            solution = (mj, dj)
            solutions.append(solution)

        return solutions

    def prove(self, solutions):
        # Generate proofs for the solutions
        proofs = [solution[1] for solution in solutions]  # Assume each solution has a dj part which is used as proof
        return proofs

    def verify(messages, proofs, commitments):
        # Verify the proofs
        for message, proof, commitment in zip(messages, proofs, commitments):
            m_com = {"message": mj, "d_j": int(proof)}
            if SHA2.hash(pack_dict(m_com)) == commitment:
                return True  # Proof is accepted
            else:
                return False  # Proof is rejected



In [814]:
# Example usage of the TimeLockedPuzzles class.

lam = 128  # Security parameter (should be at least 2048 for real scenarios)
delta = 5  # Time intervals after each message when the next should be revealed
messages = ["Secret1", "World"]  # Messages to be encrypted
z = len(messages)  # Number of puzzles/messages


# Initialize the TimeLockedPuzzles instance
tlp_instance = TimeLockedPuzzles(lam, delta, z)
print("Public Key:", tlp_instance.pk)
print("Secret Key:", tlp_instance.sk)
print("Messages:", messages)

# Generate puzzles
puzzles, commitments = tlp_instance.gen_puzzle(messages)
print("Puzzles generated.")

# Solve the puzzles
solutions = tlp_instance.solve_puzzle()
messages_dec = [s[0] for s in solutions]
assert messages == messages_dec, "Incorrect Decryption!"
print("Messages decrypted correctly.")

# Generate proofs for the solutions
proofs = tlp_instance.prove(solutions)
print("Proofs generated.")

# Verify all proofs
TimeLockedPuzzles.verify(messages_dec, proofs, commitments)
print("All proofs verified successfully!")


Public Key: {'N': 8451152962915761537992758219333932499, 'T': 50000, 'r': 4451172597240317838079427869312691627, 'aux': 'aux'}
Secret Key: {'q1': 5993361651980761111, 'q2': 1410085600311257509, 'a': 7515911595600146107146176878960470856, 'k': [615547487649126992458932822744736401], 'r': [4451172597240317838079427869312691627], 'd': [2795308964437964614617908583524828926, 5233740850050584446504787498665708135]}
Messages: ['Secret1', 'World']
Puzzles generated.
Messages decrypted correctly.
Proofs generated.
All proofs verified successfully!


# GC-TLP

In [635]:
class GCTLP:
    def __init__(self, lam, delta, S, z):
        # lam: security parameter
        # delta: a list of time period for how much longer the message must remain private after the previous was decrypted
        # S: maximum number of squaring modulo N per second
        # z: number of puzzles

        # (0) Check that the amount of time intervals provided `len(delta)` is consistent with the amount of puzzles `z`
        assert len(delta) == z, "The number of time intervals provided `delta` must be consistent with the amount of puzzles `z`"
        self.z = z
        
        # (a) Compute N = q1*q2 where q1 and q2 are large randomly chosen prime numbers
        self.q1 = random_prime(2^(lam//2), proof=False)
        self.q2 = random_prime(2^(lam//2), proof=False)
        while self.q1 == self.q2:
            self.q2 = random_prime(2^(lam//2), proof=False)
        self.N = self.q1 * self.q2
        self.phi_N = (self.q1 - 1) * (self.q2 - 1)
        
        # (b) Set Tj as the total number of squaring needed to decrypt an encrypted message mj
        self.T = [S * delta_j for delta_j in delta]
        
        # (c) Compute values aj for all j
        # Note: we can optimise by reusing the computations of the previous Tj
        self.a = [power_mod(2, Tj, self.phi_N) for Tj in self.T]
        
        # (d) Choose z fixed size random generators rj, and z random keys kj for symmetric-key encryption
            

        omega1 = lam # lam//2 + lam//2
        omega2 = lam 
        # As 256 bits is sufficient for good symetric encryption, if lam is more than 256 we limit k to 256
        key_lam = 256 if lam > 256 else lam
        k = ZZ.random_element(2^key_lam)
        
        self.r = [ZZ.random_element(1, self.N) for _ in range(z)]
        self.k = [ZZ.random_element(2^key_lam) for _ in range(z)]
        self.d = [ZZ.random_element(2^omega2) for _ in range(z)]
        
        # (e) Set the public key and secret key
        aux = 'aux'  # Placeholder for cryptographic hash function description and size of random values
        self.pk = {'aux': aux, 'N': self.N, 'T': self.T, 'r': self.r[0], 'omega1': omega1, 'omega2': omega2}
        self.sk = {'q1': self.q1, 'q2': self.q2, 'a': self.a, 'k': self.k, 'r': self.r, 'd': self.d}

    def gen_puzzle(self, messages):
        assert self.sk != [], "Please initialize the GCTLP first!"

        z = len(messages)
        
        # Encrypt the messages, starting with j = z, in descending order
        self.puzzles = []
        self.commitments = []

        for j in reversed(range(1, len(messages) + 1)):
            # Generate pkj and skj for each puzzle
            pkj = (self.N, self.T[j - 1], self.r[j - 1])
            skj = (self.q1, self.q2, self.a[j - 1], self.k[j - 1])

            # Generate puzzle
            m_enc = {"message": messages[j - 1], "d_j": int(self.d[j - 1]), "r_j": None}
            
            if j != z:
                m_enc["r_j"] = int(self.r[j])            

            puzzle = TLP.gen_puzzle(pack_dict(m_enc), pkj, skj)
            TLP.solve(puzzle, pkj)
            self.puzzles.append(puzzle)

            # Commit to each message with a cryptographic hash function
            m_com = {"message": messages[j - 1], "d_j": int(self.d[j - 1])}
            gj = SHA2.hash(pack_dict(m_com))  # Placeholder for cryptographic hash function

            self.commitments.append(gj)

        # Return the vector of puzzles and commitments
        return list(reversed(self.puzzles)), list(reversed(self.commitments))

    def solve_all(pk, puzzles):
        # Decrypt the messages in ascending order
        solutions = []
        messages = []
        u = None
        z = len(pk['T'])
        
        for j, puzzle in enumerate(puzzles, start=1):
            if j == 1:
                rj = pk['r']
            else:
                rj = u

            # Set pkj
            pkj = (pk['N'], pk['T'][j - 1], rj)

            # Solve the puzzle
            xj = TLP.solve(puzzle, pkj)
            
            # Parse the solution
            m_unpacked = unpack_dict(xj)
            if j < z:
                mj, dj, rj_plus_one = m_unpacked["message"], m_unpacked["d_j"],  m_unpacked["r_j"]
                u = rj_plus_one
            else:
                mj, dj = m_unpacked["message"], m_unpacked["d_j"]

            # Store the solution and message
            solutions.append((mj, dj))
            messages.append(mj)

        return solutions, messages


    def solve_next(pk, puzzles):
        # Decrypt the next message
        u = None
        z = len(pk['T'])

        assert z != 0, "Nothing to solve!"
        
        rj = pk['r']

        # Set pkj
        pkj = (pk['N'], pk['T'][0], rj)
        puzzle = puzzles[0]

        # Solve the puzzle
        xj = TLP.solve(puzzle, pkj)
            
        # Parse the solution
        m_unpacked = unpack_dict(xj)
        if z != 1:
            mj, dj, rj_plus_one = m_unpacked["message"], m_unpacked["d_j"],  m_unpacked["r_j"]
            u = rj_plus_one
        else:
            mj, dj = m_unpacked["message"], m_unpacked["d_j"]

        # Store the solution and message
        solution = (mj, dj)
        message = mj 

        pk_new = pk
        pk_new['r'] = u
        pk_new['T'].pop(0)
        puzzles.pop(0)

        return solution, message, pk, puzzles


    def prove(solutions):
        proofs = []
        for s in solutions:
            mj, dj = s
            # Prove that the solution is correct (for simplicity, we use the random value d as the proof)
            proofs.append(dj)
        return proofs

    def verify(mj, proof, commitment):
        # Verify the proof (in a real system, this would involve cryptographic operations)
        # Here we just check if the commitment matches the hash of mj and proof
        m_com = {"message": mj, "d_j": int(proof)}
        gj = SHA2.hash(pack_dict(m_com))  # Placeholder for cryptographic hash function
        if gj == commitment:
            return True  # Proof is accepted
        else:
            return False  # Proof is rejected



### Example usage:

In [636]:
lam = 256  # Security parameter (should be at least 2048 for real scenarios)
S = 10^4  # Maximum number of squaring modulo N per second


delta = [5, 4, 3, 2, 50]  # Time periods after each message when the next should be revealed
z = 5  # Number of puzzles/messages
messages = ["Hello!", "My", "Name", "is", "John!"] # Messages

# Initialize the GCTLP instance
gctlp_instance = GCTLP(lam, delta, S, z)
print("Public Key:", gctlp_instance.pk)
print("Secret Key:", gctlp_instance.sk)
print("Mesages:", messages)

# Select a random message


# Generate Puzzle 
p, commitments = gctlp_instance.gen_puzzle(messages)
pk = gctlp_instance.pk

# Solve the firt two puzzles seperately 
solution_one, message_one, pk, p = GCTLP.solve_next(pk, p)
assert message_one == messages[0],  "Incorrectly solved first message"
solution_two, message_two, pk, p = GCTLP.solve_next(pk, p)
assert message_two == messages[1],  "Incorrectly solved second message"

# Solve the rest of the Puzzle
solutions, solved_messages = GCTLP.solve_all(pk, p)

assert messages[2:] == solved_messages, "Incorrectly solved all messages:\nexpected: {}\nrecieved: {}".format(messages,solved_messages)
print("All messages solved sussessfully!")

# Merge solutions obained from individual steps:
solutions = [solution_one] + [solution_two] + solutions
solved_messages = [message_one] + [message_two] + solved_messages
assert messages == solved_messages, "Something did not add up over the solved messages"

# Prove
proofs = GCTLP.prove(solutions)

# Verify all Proofs
for j, (mj, proof, commitment) in enumerate(zip(solved_messages, proofs, commitments), start=1):
    assert GCTLP.verify(mj, proof, commitment), "Failed to verify at position {}".format(j)

print("Everything verified successfully!")

Public Key: {'aux': 'aux', 'N': 1773511781373259697859645028342125082454194194700104363876413078238650813809, 'T': [50000, 40000, 30000, 20000, 500000], 'r': 1626333349410856463229043625342604768110164310608695392094584747230863989091, 'omega1': 256, 'omega2': 256}
Secret Key: {'q1': 6059271902891524312007659194638687027, 'q2': 292693876392463628888327621090405029067, 'a': [1059807271669194719854782324469827673754479869884402926473841201525927375632, 1007908771366805014755071263433758289752445087395214715367328088507207568176, 1700370423069252287961272723287987274957764685152629436049830557724434342180, 1380787107076729120535803280014449066057586725404924795699368363473628958368, 1055964556509800640518822510500966570859509150305625956078467371565039337244], 'k': [20413298193709824565323544323366760661241249137393611439838649093854667038965, 2800807105535599801262144725528293755207109451920279019683335171170077201959, 490493492080915472468975960688351399075834530364261652337446118419281

# Effecietly Delegatable Time Lock Puzzle (ED-TLP)

### Web3 

In [643]:
BLOCKCHAIN = True

In [740]:
# Web3 Setup
from web3 import Web3
from web3.middleware import geth_poa_middleware

# Placeholder values for private keys of the Client, Server, and TPH
PRIVATE_KEY_CLIENT_THP = 'YOUR_PRIVATE_KEY_1'
PRIVATE_KEY_SERVER = 'YOUR_PRIVATE_KEY_2'
PRIVATE_KEY_SERVER_TPH = 'YOUR_PRIVATE_KEY_3'

# Ethereum node RPC URL (use the appropriate URL for the testnet you're using, e.g., Rinkeby, Ropsten)
RPC_URL = 'YOUR_RPC_URL_SEPOLIA'

# Setup web3 instance
w3 = Web3(Web3.HTTPProvider(RPC_URL))

# If you're using a testnet like Rinkeby that uses Proof-of-Authority, you may need the middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=int(0))

# Addresses of the smart contract and participants (placeholders)
CONTRACT_ADDRESS = '0x9ac103735E60c4D3445A068162e943f6784bE104'
CONTRACT_ABI = """
[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"puzzleSetId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"puzzleId","type":"uint256"},{"indexed":false,"internalType":"address","name":"disputer","type":"address"}],"name":"DisputeRaised","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"puzzleSetId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"puzzleId","type":"uint256"},{"indexed":false,"internalType":"address","name":"solver","type":"address"}],"name":"PaymentIssued","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"puzzleSetId","type":"uint256"}],"name":"PuzzleSetCreated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"puzzleSetId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"puzzleId","type":"uint256"},{"indexed":false,"internalType":"address","name":"solver","type":"address"}],"name":"PuzzleSolved","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"puzzleSetId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"puzzleId","type":"uint256"},{"indexed":false,"internalType":"address","name":"server","type":"address"}],"name":"WithdrawalAllowed","type":"event"},{"inputs":[{"internalType":"uint256","name":"puzzleSetId","type":"uint256"},{"internalType":"uint256","name":"puzzleId","type":"uint256"},{"internalType":"bool","name":"isValidSolution","type":"bool"}],"name":"confirmPayment","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_numberOfPuzzles","type":"uint256"},{"internalType":"uint256[]","name":"_deliveryTimes","type":"uint256[]"},{"internalType":"uint256[]","name":"_coins","type":"uint256[]"},{"internalType":"address","name":"solver","type":"address"}],"name":"createPuzzleSet","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"disputeDuration","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"puzzleSetId","type":"uint256"},{"internalType":"bytes[]","name":"p","type":"bytes[]"},{"internalType":"bytes32[]","name":"g","type":"bytes32[]"}],"name":"initializePuzzle","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"puzzleSetCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"puzzleSets","outputs":[{"internalType":"uint256","name":"numberOfPuzzles","type":"uint256"},{"internalType":"uint256","name":"startTime","type":"uint256"},{"internalType":"address","name":"client","type":"address"},{"internalType":"address","name":"server","type":"address"},{"internalType":"uint256","name":"deposit","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"puzzleSetId","type":"uint256"},{"internalType":"uint256","name":"puzzleId","type":"uint256"}],"name":"resolveDispute","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"puzzleSetId","type":"uint256"},{"internalType":"uint256","name":"puzzleId","type":"uint256"},{"internalType":"bytes","name":"solution","type":"bytes"},{"internalType":"bytes","name":"proof","type":"bytes"}],"name":"submitSolution","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"puzzleSetId","type":"uint256"},{"internalType":"uint256","name":"puzzleId","type":"uint256"}],"name":"withdrawUnclaimedCoins","outputs":[],"stateMutability":"nonpayable","type":"function"}]
                """
# Convert private keys to addresses
CLIENT_ADDRESS = w3.eth.account.from_key(PRIVATE_KEY_CLIENT_THP).address
SERVER_ADDRESS = w3.eth.account.from_key(PRIVATE_KEY_SERVER).address
TPH_ADDRESS = w3.eth.account.from_key(PRIVATE_KEY_SERVER_TPH).address

In [741]:
class Web3Helper:
    def __init__(self, w3_instance, contract_address, contract_abi, private_key):
        self.w3 = w3_instance
        self.contract_address = contract_address
        self.private_key = private_key
        self.account = self.w3.eth.account.from_key(private_key)
        self.contract = self.w3.eth.contract(address=self.contract_address, abi=contract_abi)


    def send_transaction(self, function_call, value=int(0), gas=None):
        tx = function_call.build_transaction({
            'gasPrice': self.w3.to_wei('1', 'gwei'),
            'nonce': self.w3.eth.get_transaction_count(self.account.address),
            'value': value
        })
        
        # Note: for some reason the direct gas estimation fails, hence we allow to overwrite it manually.
        if gas != None:
            tx['gas'] = int(gas)
            
        signed_tx = self.w3.eth.account.sign_transaction(tx, self.private_key)
        tx_hash = self.w3.eth.send_raw_transaction(signed_tx.rawTransaction)
        return tx_hash

    def wait_for_transaction_receipt(self, tx_hash):
        """Waits for the transaction to be mined and returns the transaction receipt."""
        tx_receipt = self.w3.eth.wait_for_transaction_receipt(tx_hash)
        if tx_receipt:
            return tx_receipt


    def get_puzzle_set_counter(self):
        """Get the current puzzle set counter."""
        return self.contract.functions.puzzleSetCounter().call()


In [742]:
LAMBDA = 256  # For demonstration purposes; use 2048 or higher in production
S_DEFAULT = 10**4  # Maximum number of squaring modulo N per second

In [743]:
ToC_DEFAULT = 0 # Consensus time, we ignore it for now
Y_DEFAULT = 0 # Network delay, we ignore it for now
T_0_DEFAULT = 0 # When the puzzle is released. We set it to 0 as we delegate its setup to the Smart Contract.

In [744]:
DEFAULT_AUX = "AMD850_12"

def calculate_default_cedg(ToC, S, delta_j, aux=DEFAULT_AUX):
    """
    Calculate the default Cost-Effective Delivery Guarantee (CEDG).
    
    Parameters:
        ToC (float): Time of Computation for the puzzle.
        S (float): Server's number of squaring modulo N per second.
        delta_j (float): Delivery time for the puzzle.
        aux (str): Auxiliary information (default: "AMD850_12").
        
    Returns:
        float: Cost-Effective Delivery Guarantee.
    """
    return S * 10 * delta_j  # By default, give ten times more than to the best possible server


In [745]:
class Client:

    def __init__(self):
        
        self.setupKey()
        
    def setupKey(self):
        """
        Generate a symmetric key for the client.
        """
        # Generate a symmetric key k (for simplicity, using a random integer)
        key_size = AES.key_size(LAMBDA) # Selects a suitable key size in bits for AES based on security parameter
        self.csk = ZZ.random_element(2**key_size)  # Insecure random
        self.csk = int(self.csk).to_bytes(key_size // 8, byteorder='big')


    def delegate(self, messages, t_0=T_0_DEFAULT):
        """
        Encrypt messages using the client's symmetric key so that they can be securely delegated.

        Parameters:
            messages (list): List of messages to encrypt.
            t_0 (int): Time the puzzle is released.

        Returns:
            tuple: Encrypted messages and time the puzzle is released.
        """
        assert self.csk is not None, "You must first initialize the Client!"

        messages_enc = []

        for m in messages:
            m_enc = AES.encrypt(m, self.csk)
            messages_enc.append(m_enc)

        return messages_enc, t_0

In [746]:
class Server:

    def __init__(self, eth_private_key, csk, contract_address=CONTRACT_ADDRESS, contract_abi=CONTRACT_ABI):
        self.csk = csk
        
        self.eth_helper = Web3Helper(w3, contract_address, contract_abi, eth_private_key)

    def delegate(self, deltas, coins, solver=None, t_0=T_0_DEFAULT, ToC=ToC_DEFAULT, Y=Y_DEFAULT, CEDG=calculate_default_cedg, S=S_DEFAULT, aux=DEFAULT_AUX):
        """
        Delegate puzzle setup to the Third Helping Party (THP).

        Parameters:
            deltas (list): List of delivery times for each puzzle.
            coins (list): List of coins for each puzzle.
            solver (address): Address of the puzzle solver (optional).
            t_0 (int): Time the puzzle is released (default: T_0_DEFAULT).
            Y (int): Default consensus time (default: Y_DEFAULT).
            CEDG (function): Function to calculate Cost-Effective Delivery Guarantee (default: calculate_default_cedg).
            S (int): Maximum number of squaring modulo N per second (default: S_DEFAULT).
            aux (str): Auxiliary information (default: DEFAULT_AUX).
            
        Returns:
            tuple: Phi values and puzzle set ID.
        """
        T = []
        phi = []
        for j, delta_j in enumerate(deltas, start=1):
            phi_j = CEDG(ToC, S, delta_j, aux)
            phi.append(phi_j)

            if j == 1:
                T_j = t_0
            else:
                T_j = T[-1]
                
            T_j = delta_j + phi_j + Y
            T.append(T_j)



        if BLOCKCHAIN:
    
            # Prepare inputs for the smart contract
            z = len(T)
            solver_address = solver if solver else '0x0000000000000000000000000000000000000000'
            T = [int(T_j) for T_j in T]
            coins = [int(coin) for coin in coins]
    
    
            # Prepare the function call for the smart contract
            setup_tx = self.eth_helper.contract.functions.createPuzzleSet(z, T, coins, solver_address) 
    
    
            # Send the transaction with the correct value
            tx_hash = self.eth_helper.send_transaction(setup_tx, value=sum(coins), gas=500000)
            print(f"Setup PuzzleSet transaction sent. TX hash: {tx_hash.hex()}")
    
            # Wait for the transaction to be mined to retrieve the puzzleSetId
            receipt = self.eth_helper.wait_for_transaction_receipt(tx_hash)
            print("Transaction has been mined successfully.")
    
            # Access the puzzleSetCounter
            puzzle_set_counter = self.eth_helper.get_puzzle_set_counter()
            puzzle_set_id = puzzle_set_counter - 1  # Assuming the counter increments after the transaction
            print(f"The current PuzzleSet ID is: {puzzle_set_id}")
            
        else:
            
            puzzle_set_id = 0
            
        return phi, puzzle_set_id
        print("Successfully Setted Up the Puzzle!")

    def verify(self, puzzle_id, puzzle_set_id, message_enc_j, proof_j, commit_j):
        """
        Verify the solution of a puzzle.

        Parameters:
            pk (str): Public puzzle information.
            puzzle_id (int): Puzzle ID.
            puzzle_set_id (int): Puzzle set ID in a smart contarct.
            message_enc (bytes): Encrypted message.
            proof_j (bytes): Proof for Puzzle with ID j.
            commit_j (bytes): Commitment for Puzzle with ID j.
        """

        valid = GCTLP.verify(message_enc_j, proof_j, commit_j)

        print(f"It is {'valid' if valid else 'invalid'}")

        if BLOCKCHAIN:
    
            # Call the smart contract function confirmPayment
            verify_tx = self.eth_helper.contract.functions.confirmPayment(int(puzzle_set_id), int(puzzle_id), valid)
            tx_hash = self.eth_helper.send_transaction(verify_tx)
            receipt = self.eth_helper.wait_for_transaction_receipt(tx_hash)
    
            if receipt.status == 1:
                print(f"Solution for puzzle {puzzle_id} in set {puzzle_set_id} verified successfully.")
            else:
                print(f"Solution for puzzle {puzzle_id} in set {puzzle_set_id} failed to verify.")


    def withdraw_dispute(self, puzzle_id, puzzle_set_id):
        assert BLOCKCHAIN, "This function in unavailable without Blockchain"
        
        """
        Withdraw dispute.

        Parameters:
            puzzle_id (int): Puzzle ID.
            puzzle_set_id (int): Puzzle set ID.
        """
        # Call the smart contract function withdrawUnclaimedCoins
        withdraw_tx = self.eth_helper.contract.functions.withdrawUnclaimedCoins(puzzle_set_id, puzzle_id)
        tx_hash = self.eth_helper.send_transaction(withdraw_tx)
        receipt = self.eth_helper.wait_for_transaction_receipt(tx_hash)

        if receipt.status == 1:
            print(f"Withdrawal for puzzle {puzzle_id} in set {puzzle_set_id} successful.")
        else:
            print(f"Withdrawal for puzzle {puzzle_id} in set {puzzle_set_id} failed.")


    def decrypt(self, messages_enc):

        messages_dec = []

        for m_enc in messages_enc:
            m_dec = AES.decrypt(m_enc, self.csk)
            messages_dec.append(m_dec)

        return messages_dec
        
        

In [747]:
class Client_TPH:

    def __init__(self, deltas, z, eth_private_key, S=S_DEFAULT, contract_address=CONTRACT_ADDRESS, contract_abi=CONTRACT_ABI):
        self.gctlp = GCTLP(LAMBDA, deltas, S, z)
        self.pk = self.gctlp.pk
        
        # Setup Web3 helper with private key and contract ABI
        self.eth_helper = Web3Helper(w3, contract_address, contract_abi, eth_private_key)


    def gen_puzzle(self, messages_enc, puzzle_set_id, t_0=T_0_DEFAULT):
        p, g = self.gctlp.gen_puzzle(messages_enc)

        if BLOCKCHAIN:

            # Ensure the p and g are formatted as lists of bytes32 for Solidity
            # Depending on how GCTLP generates p and g, you might need to format or cast these values.
            # p_call = [Web3.to_bytes(hexstr=Web3.to_hex(text=str(item))) for item in p]
            g_call = [HexBytes(item) for item in g]
                
            # Prepare the function call to initialisePuzzle
            init_puzzle_tx = self.eth_helper.contract.functions.initializePuzzle(
                int(puzzle_set_id),
                [HexBytes(b'0'*32)] * len(g_call),
                g_call
            )
    
            # Send the transaction
            tx_hash = self.eth_helper.send_transaction(init_puzzle_tx, gas=200000)
            print(f"Initialise Puzzle transaction sent. TX hash: {tx_hash.hex()}")
    
            # Wait for the transaction to be mined
            receipt = self.eth_helper.wait_for_transaction_receipt(tx_hash)
            print("Initialise Puzzle transaction has been mined successfully.")

        return p, g  # Optionally return p, g if needed elsewhere

In [748]:
class Server_TPH:

    def __init__(self, pk, p, phi, coins, puzzle_set_id,  eth_private_key, aux=DEFAULT_AUX, contract_address=CONTRACT_ADDRESS, contract_abi=CONTRACT_ABI):

        if Server_TPH._decide_to_solve(phi, aux, coins):   
            self.pk = pk
            self.p = deepcopy(p)
            self.puzzle_set_id = puzzle_set_id
            self.solved = 0
            
            self.solutions = []
            self.messages_enc = []
        
        # Setup Web3 helper with private key and contract ABI
        self.eth_helper = Web3Helper(w3, contract_address, contract_abi, eth_private_key)


    def _decide_to_solve(phi, aux, coins):
        # Placeholder logic to decide whether to solve based on auxiliary data and coin value
        return True  # Assume we decide to solve for now
        
    def test_solve(self):
        # We do not send to blockchain in this version
        # This function does not update the internal state
        
        solutions, messages_enc = GCTLP.solve_all(self.pk, self.p)
        return (solutions, messages_enc)

    def solve_iteratively(self, puzzle_set_id, aux=DEFAULT_AUX, puzzle_limit=None):
        # We send each solved solution to blockchain iteratively
        
        # Allow to select how many puzzles to solve in a single go, if puzzle_limit = 1 solve only one puzzle
        puzzle_limit = puzzle_limit if puzzle_limit != None else len(self.p)
        
        for j in range(self.solved, puzzle_limit + self.solved):
            solution, message_enc, self.pk, self.p = GCTLP.solve_next(self.pk, self.p)
            
            self.solutions.append(solution)
            self.messages_enc.append(message_enc)

            proof = self.prove([solution])

            if BLOCKCHAIN:

                # Register solution to the blockchain
                register_solution_tx = self.eth_helper.contract.functions.submitSolution(
                    int(puzzle_set_id),
                    int(j),
                    Web3.to_bytes(hexstr=Web3.to_hex(text=str(solution))),
                    Web3.to_bytes(hexstr=Web3.to_hex(text=str(proof)))
                )
    
                # Send the transaction
                tx_hash = self.eth_helper.send_transaction(register_solution_tx, gas=600000)
                print(f"Register Solution transaction sent. TX hash: {tx_hash.hex()}")
    
                # Optionally wait for the transaction to be mined
                receipt = self.eth_helper.wait_for_transaction_receipt(tx_hash)
                print("Register Solution transaction has been mined successfully.")

        
        return (self.solutions, self.messages_enc)

    def prove(self, solutions):
        # Generate proof using the GCTLP protocol
        proof = GCTLP.prove(solutions)
        return proof


    def dispute(self, pk, puzzle_id, puzzle_set_id):
        assert BLOCKCHAIN, "This function in unavailable without Blockchain"

       # Raise a dispute for a given puzzle at the blockchain
        dispute_tx = self.eth_helper.contract.functions.raiseDispute(
            int(puzzle_set_id),
            int(puzzle_id)
        )

        # Send the dispute transaction
        tx_hash = self.eth_helper.send_transaction(dispute_tx)
        print(f"Dispute transaction sent. TX hash: {tx_hash.hex()}")

        # Wait for the transaction to be mined
        receipt = self.eth_helper.wait_for_transaction_receipt(tx_hash)
        print("Dispute transaction has been mined successfully.")
    

In [749]:
messages = ["Hello!", "World!"]
client = Client()
messages_enc, _  = client.delegate(messages)
csk = client.csk
# Map messages to a str format
messages_enc = [m_enc.hex() for m_enc in messages_enc]

In [754]:
deltas = [2, 2]
coins = [100, 100]
number_messages = len(deltas)
server = Server(PRIVATE_KEY_SERVER, csk)
phi, puzzle_set_id = server.delegate(deltas, coins)

Setup PuzzleSet transaction sent. TX hash: 0x1a9b51fcd32d3a810cbd3915be3c76220b34f6c23f56bbfa288d3c374f2cbeb4
Transaction has been mined successfully.
The current PuzzleSet ID is: 1


In [755]:
from hexbytes import HexBytes
client_helper = Client_TPH(deltas, number_messages, PRIVATE_KEY_SERVER)
pk = client_helper.pk
p, g = client_helper.gen_puzzle(messages_enc, puzzle_set_id)

Initialise Puzzle transaction sent. TX hash: 0xb10595d5372256a99c2b1150b490ba4796da8b7f1b555f80a751c2e7a67fbb97
Initialise Puzzle transaction has been mined successfully.


In [756]:
server_helper = Server_TPH(pk, p, phi, coins, puzzle_set_id, PRIVATE_KEY_SERVER_TPH)
solutions, messages_enc = server_helper.solve_iteratively(puzzle_set_id)


puzzle_id = 0
proofs = server_helper.prove(solutions)

Register Solution transaction sent. TX hash: 0x0afb8a006a969f17d15f35fbecb49830212c30375dec5ef8f638d92523026531
Register Solution transaction has been mined successfully.
Register Solution transaction sent. TX hash: 0x8d57eb8038c78ad90bd5bc429e6455da3daae3b13163940a5dd72af5495c002d
Register Solution transaction has been mined successfully.


In [757]:
server.verify(puzzle_id, puzzle_set_id, messages_enc[puzzle_id], proofs[puzzle_id], g[puzzle_id])

It is valid
Solution for puzzle 0 in set 1 verified successfully.


In [758]:
# Decode bytes from hex strings
messages_enc = [bytes.fromhex(m_enc) for m_enc in messages_enc]
server.decrypt([bytes(m_enc) for m_enc in messages_enc])

['Hello!', 'World!']